In [32]:
##Logan Lauton
##Exploratory Stats

##importing required packages for this exploratory stats
import requests
import oracledb
import getpass
import cx_Oracle
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from sklearn.linear_model import LinearRegression

##connecting to the db as we did in grab_data.ipynb
os.environ.get('TNS_ADMIN')
connection = oracledb.connect(
    user="admin",
    password="aybM6CFL9gykHEKn",
    dsn = "ltl3y0m4d7of29l1_high",
    config_dir="./config",
    wallet_location="./config",
    wallet_password="phbUHx6CMjxQj75A")

##print to ensure that connection is successful
print("Successfully connected to Oracle Database")

cursor = connection.cursor()

##selecting everything from the calls table and creating a df
rs1 = cursor.execute("SELECT * FROM CALLS")
df1 = pd.DataFrame(rs1.fetchall())
rs2 = cursor.execute("SELECT * FROM CENSUS")
df2 = pd.DataFrame(rs2.fetchall())
rs3 = cursor.execute("SELECT * FROM HISTORIC_REAL_ESTATE")
df3 = pd.DataFrame(rs3.fetchall())

##print to ensure import of data is successful
print('Successfully imported tables from Oracle Database')

##properly naming the columns
df1.columns = ['Call Number' , 'Date/Time' , 'Address' , 'District' , 'Nature of Call' , 'Status' , 'Latitude' , 'Longitude', "Tract"]
df2.columns = ['GEOID' , 'Tract' , 'Geometry' , 'Estimated Population' , 'Male Population' , 'Female Population' , 'Median Age' , 'Median Income', 'White Population', 'Black Population', 'Native Population', 'Hispanic Population', 'Income Poverty Level']
##cleaning df1
df1 = df1.sort_values(by='Date/Time', ascending=False)
df1[["Tract"]] = df1[["Tract"]].apply(pd.to_numeric)

##cleaing df2
df2[["Tract","Estimated Population", "Male Population" ,"Female Population", "Median Age", "Median Income","White Population", "Black Population", "Native Population", "Hispanic Population","Income Poverty Level"]] = df2[["Tract","Estimated Population", "Male Population" ,"Female Population", "Median Age", "Median Income","White Population", "Black Population", "Native Population", "Hispanic Population","Income Poverty Level"]].apply(pd.to_numeric)
df2["Hispanic Population"] = df2["Estimated Population"] - (df2["White Population"]+df2["Black Population"]+df2["Native Population"])
df2["Percent White"] = round((df2["White Population"] / df2["Estimated Population"])*100,3)
df2["Percent Black"] = round((df2["Black Population"] / df2["Estimated Population"])*100,3)
df2["Percent Native"] = round((df2["Native Population"] / df2["Estimated Population"])*100,3)
df2["Percent Hispanic"] = round((df2["Hispanic Population"] / df2["Estimated Population"])*100,3)
df2["Predominent Tract Population"] = df2[['Percent White', 'Percent Black', 'Percent Native','Percent Hispanic']] \
    .idxmax(axis=1).str.replace('Percent ', '').tolist()

Successfully connected to Oracle Database
Successfully imported tables from Oracle Database


In [33]:
print("Percent White in Milwaukee(Should be 39.9):")
print(round((sum(df2["White Population"])/sum(df2["White Population"]+df2["Black Population"]+df2["Native Population"]+df2["Hispanic Population"]))*100,3))
print("Percent Black in Milwaukee(Should be 39.4):")
print(round((sum(df2["Black Population"])/sum(df2["White Population"]+df2["Black Population"]+df2["Native Population"]+df2["Hispanic Population"]))*100,3))
print("Percent Native in Milwaukee(0.6):")
print(round((sum(df2["Native Population"])/sum(df2["White Population"]+df2["Black Population"]+df2["Native Population"]+df2["Hispanic Population"]))*100,3))
print("Percent Hispanic in Milwaukee(Should be 19.9):")
print(round((sum(df2["Hispanic Population"])/sum(df2["White Population"]+df2["Black Population"]+df2["Native Population"]+df2["Hispanic Population"]))*100,3))

print("The percents in the () are given from https://www.census.gov/quickfacts/milwaukeecitywisconsin")

Percent White in Milwaukee(Should be 39.9):
56.522
Percent Black in Milwaukee(Should be 39.4):
26.857
Percent Native in Milwaukee(0.6):
0.529
Percent Hispanic in Milwaukee(Should be 19.9):
16.092
The percents in the () are given from https://www.census.gov/quickfacts/milwaukeecitywisconsin


In [34]:
df2

,GEOID,Tract,Geometry,Estimated Population,Male Population,Female Population,Median Age,Median Income,White Population,Black Population,Native Population,Hispanic Population,Income Poverty Level,Percent White,Percent Black,Percent Native,Percent Hispanic,Prodominent Tract Population
0,008700,55079008700,POLYGON ((-65371.33323294844 4791454.558569728...,890.0,403.0,487.0,32.9,13782.0,0.0,890.0,0.0,0.0,890.0,0.000,100.000,0.000,0.000,Black
1,008800,55079008800,POLYGON ((-65976.74531288422 4791487.360095243...,1620.0,795.0,825.0,32.5,16713.0,90.0,1498.0,4.0,28.0,1620.0,5.556,92.469,0.247,1.728,Black
2,008500,55079008500,POLYGON ((-64585.292365803034 4791763.78518607...,1540.0,756.0,784.0,29.5,19546.0,6.0,1517.0,17.0,0.0,1540.0,0.390,98.506,1.104,0.000,Black
3,009200,55079009200,POLYGON ((-68392.08530371182 4790901.985810455...,1630.0,807.0,823.0,30.7,23385.0,537.0,759.0,8.0,326.0,1602.0,32.945,46.564,0.491,20.000,Black
4,008900,55079008900,POLYGON ((-66590.0980055522 4791541.1265165135...,1548.0,724.0,824.0,21.4,12663.0,95.0,1268.0,0.0,185.0,1444.0,6.137,81.912,0.000,11.951,Black
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,180400,55079180400,POLYGON ((-60159.98100179259 4776059.551584941...,2581.0,1325.0,1256.0,47.2,35019.0,2191.0,151.0,36.0,203.0,2562.0,84.890,5.850,1.395,7.865,White
289,180500,55079180500,POLYGON ((-62146.70780221652 4776133.915462926...,4430.0,1965.0,2465.0,56.4,35452.0,4103.0,40.0,10.0,277.0,4411.0,92.619,0.903,0.226,6.253,White
290,185100,55079185100,POLYGON ((-61325.541799318045 4781700.73889351...,4779.0,2290.0,2489.0,53.3,31083.0,4302.0,103.0,43.0,331.0,4761.0,90.019,2.155,0.900,6.926,White
291,185200,55079185200,POLYGON ((-62326.85597527737 4780182.084367800...,4821.0,2342.0,2479.0,47.6,32330.0,4064.0,138.0,0.0,619.0,4777.0,84.298,2.862,0.000,12.840,White


In [35]:
resultdf=df1.merge(df2,how="left",on="Tract")
resultdf

,Call Number,Date/Time,Address,District,Nature of Call,Status,Latitude,Longitude,Tract,GEOID,...,White Population,Black Population,Native Population,Hispanic Population,Income Poverty Level,Percent White,Percent Black,Percent Native,Percent Hispanic,Prodominent Tract Population
0,230910713,04/01/2023 12:59:45 PM,"2323 N MARTIN L KING JR DR,MKE",5,BUSINESS CHECK,Assignment Completed,43.045775,-87.914421,5.507901e+10,014100,...,241.0,1167.0,0.0,81.0,1489.0,16.185,78.375,0.000,5.440,Black
1,230910712,04/01/2023 12:59:41 PM,"2357 N 27TH ST,MKE",3,TRAFFIC STOP,City Citation(s) Issued,43.061626,-87.947602,5.507901e+10,009900,...,21.0,1083.0,0.0,89.0,1193.0,1.760,90.780,0.000,7.460,Black
2,230910060,04/01/2023 12:59:41 AM,"333 W JUNEAU AV,MKE",1,ENTRY,Service in Progress,43.045353,-87.915614,5.507919e+10,186300,...,2330.0,1630.0,8.0,456.0,2356.0,52.667,36.844,0.181,10.307,White
3,230901607,04/01/2023 12:59:01 AM,"3352 N 10TH ST,MKE",5,BATTERY,Service in Progress,43.080117,-87.923350,5.507901e+10,006800,...,100.0,2118.0,9.0,32.0,2131.0,4.427,93.758,0.398,1.417,Black
4,230910700,04/01/2023 12:58:57 PM,"840 N 24TH ST,MKE",3,WELFARE CITIZEN,Advised,43.041360,-87.942658,5.507901e+10,014800,...,1215.0,1182.0,30.0,289.0,2299.0,44.735,43.520,1.105,10.641,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40992,230310979,01/31/2023 02:18:31 PM,"1002 W MAPLE ST,MKE",2,TRBL W/SUBJ,Assignment Completed,43.011106,-87.924340,5.507902e+10,016700,...,1661.0,463.0,53.0,1143.0,3320.0,50.030,13.946,1.596,34.428,White
40993,230310974,01/31/2023 02:17:47 PM,"2300 W HIGHLAND AV,MKE",3,THREAT SCHOOL,Service in Progress,43.045270,-87.941590,5.507901e+10,014800,...,1215.0,1182.0,30.0,289.0,2299.0,44.735,43.520,1.105,10.641,White
40994,230310931,01/31/2023 02:16:59 PM,"380 E CAPITOL DR,MKE",5,WELFARE CITIZEN,Unable to Locate Complainant,43.090867,-87.905326,5.507900e+10,004400,...,678.0,2003.0,0.0,191.0,2872.0,23.607,69.742,0.000,6.650,Black
40995,230310978,01/31/2023 02:16:01 PM,"3626 W FOND DU LAC AV,MKE",7,SPECIAL ASSIGN,Service in Progress,43.076747,-87.958838,5.507900e+10,004800,...,314.0,3124.0,4.0,54.0,3488.0,8.982,89.359,0.114,1.545,Black


In [36]:
empty = resultdf[resultdf['GEOID'].isnull()]
empty

,Call Number,Date/Time,Address,District,Nature of Call,Status,Latitude,Longitude,Tract,GEOID,...,White Population,Black Population,Native Population,Hispanic Population,Income Poverty Level,Percent White,Percent Black,Percent Native,Percent Hispanic,Prodominent Tract Population
1457,230901406,03/31/2023 07:03:53 PM,6931 S HOWELL,SPD,TRAFFIC STOP,Advised,42.865506,-87.910120,5.507916e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1690,230901165,03/31/2023 04:46:18 PM,"500 E OAK ST, OAK CREEK",6,PROPERTY PICKUP,Assignment Completed,42.923150,-87.905010,5.507916e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1785,230901095,03/31/2023 04:10:25 PM,5970 S HOWELL,SPD,TRAFFIC STOP,City Citation(s) Issued,42.865506,-87.910120,5.507916e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3654,230880670,03/29/2023 11:56:13 AM,"S 6TH ST / W COLLEGE AV,MKE",6,PROPERTY PICKUP,Assignment Completed,42.930108,-87.919887,5.507916e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4890,230880837,03/29/2023 01:39:04 PM,"2250 N LYNETTE LN, BROOKFIELD",SPD,ASSIGNMENT,Service in Progress,43.061180,-88.178218,5.513320e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39242,230320659,02/01/2023 11:20:11 AM,D2,2,FOLLOW UP,Assignment Completed,46.109714,-111.166100,3.003100e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39748,230321215,02/01/2023 07:06:23 PM,"8705 N 43RD ST, BROWNDEER WI",5,BATTERY,Service in Progress,43.176117,-87.964664,5.507905e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39858,230321348,02/01/2023 06:02:30 PM,D4,4,PROBATION/PAROLE,Service in Progress,46.109714,-111.166100,3.003100e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40486,230311545,01/31/2023 09:23:22 PM,"8250 N 46TH ST, BROWN DEER",4,VIOL REST ORDER,Assignment Completed,43.168187,-87.968282,5.507905e+10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
resultdf = resultdf.dropna(subset=['GEOID'])
resultdf

,Call Number,Date/Time,Address,District,Nature of Call,Status,Latitude,Longitude,Tract,GEOID,...,White Population,Black Population,Native Population,Hispanic Population,Income Poverty Level,Percent White,Percent Black,Percent Native,Percent Hispanic,Prodominent Tract Population
0,230910713,04/01/2023 12:59:45 PM,"2323 N MARTIN L KING JR DR,MKE",5,BUSINESS CHECK,Assignment Completed,43.045775,-87.914421,5.507901e+10,014100,...,241.0,1167.0,0.0,81.0,1489.0,16.185,78.375,0.000,5.440,Black
1,230910712,04/01/2023 12:59:41 PM,"2357 N 27TH ST,MKE",3,TRAFFIC STOP,City Citation(s) Issued,43.061626,-87.947602,5.507901e+10,009900,...,21.0,1083.0,0.0,89.0,1193.0,1.760,90.780,0.000,7.460,Black
2,230910060,04/01/2023 12:59:41 AM,"333 W JUNEAU AV,MKE",1,ENTRY,Service in Progress,43.045353,-87.915614,5.507919e+10,186300,...,2330.0,1630.0,8.0,456.0,2356.0,52.667,36.844,0.181,10.307,White
3,230901607,04/01/2023 12:59:01 AM,"3352 N 10TH ST,MKE",5,BATTERY,Service in Progress,43.080117,-87.923350,5.507901e+10,006800,...,100.0,2118.0,9.0,32.0,2131.0,4.427,93.758,0.398,1.417,Black
4,230910700,04/01/2023 12:58:57 PM,"840 N 24TH ST,MKE",3,WELFARE CITIZEN,Advised,43.041360,-87.942658,5.507901e+10,014800,...,1215.0,1182.0,30.0,289.0,2299.0,44.735,43.520,1.105,10.641,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40992,230310979,01/31/2023 02:18:31 PM,"1002 W MAPLE ST,MKE",2,TRBL W/SUBJ,Assignment Completed,43.011106,-87.924340,5.507902e+10,016700,...,1661.0,463.0,53.0,1143.0,3320.0,50.030,13.946,1.596,34.428,White
40993,230310974,01/31/2023 02:17:47 PM,"2300 W HIGHLAND AV,MKE",3,THREAT SCHOOL,Service in Progress,43.045270,-87.941590,5.507901e+10,014800,...,1215.0,1182.0,30.0,289.0,2299.0,44.735,43.520,1.105,10.641,White
40994,230310931,01/31/2023 02:16:59 PM,"380 E CAPITOL DR,MKE",5,WELFARE CITIZEN,Unable to Locate Complainant,43.090867,-87.905326,5.507900e+10,004400,...,678.0,2003.0,0.0,191.0,2872.0,23.607,69.742,0.000,6.650,Black
40995,230310978,01/31/2023 02:16:01 PM,"3626 W FOND DU LAC AV,MKE",7,SPECIAL ASSIGN,Service in Progress,43.076747,-87.958838,5.507900e+10,004800,...,314.0,3124.0,4.0,54.0,3488.0,8.982,89.359,0.114,1.545,Black


In [38]:
#patrol call per tract for majority white neighborhoods vs other - check for different h0 m1==m2, with very low p value.
patroldata = resultdf[resultdf['Nature of Call'] == "PATROL"]
patroldata

,Call Number,Date/Time,Address,District,Nature of Call,Status,Latitude,Longitude,Tract,GEOID,...,White Population,Black Population,Native Population,Hispanic Population,Income Poverty Level,Percent White,Percent Black,Percent Native,Percent Hispanic,Prodominent Tract Population
23,230910068,04/01/2023 12:50:52 AM,"N HOPKINS ST / W GLENDALE AV,MKE",7,PATROL,Assignment Completed,43.100776,-87.958364,5.507900e+10,002600,...,247.0,2325.0,27.0,72.0,2663.0,9.247,87.046,1.011,2.696,Black
31,230910696,04/01/2023 12:46:25 PM,"1800 N 2ND ST,MKE",5,PATROL,Assignment Completed,43.053962,-87.912653,5.507901e+10,011400,...,1147.0,478.0,0.0,41.0,1666.0,68.848,28.691,0.000,2.461,White
42,230910054,04/01/2023 12:40:17 AM,"N 60TH ST / W BIRCH AV,MKE",4,PATROL,Assignment Completed,43.117238,-87.986212,5.507900e+10,001900,...,287.0,2797.0,0.0,226.0,3310.0,8.671,84.502,0.000,6.828,Black
71,230910032,04/01/2023 12:25:46 AM,"2745 S HOWELL AV,MKE",6,PATROL,Assignment Completed,42.994612,-87.904592,5.507902e+10,018400,...,1379.0,0.0,0.0,60.0,1425.0,95.830,0.000,0.000,4.170,White
72,230910031,04/01/2023 12:25:24 AM,"N 53RD ST / W CENTER ST,MKE",3,PATROL,Assignment Completed,43.067993,-87.979848,5.507901e+10,005900,...,889.0,2590.0,0.0,285.0,3684.0,23.618,68.810,0.000,7.572,Black
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40678,230311441,01/31/2023 06:26:38 PM,"N PALMER ST / E CHAMBERS ST,MKE",5,PATROL,Assignment Completed,43.052670,-87.909770,5.507901e+10,011400,...,1147.0,478.0,0.0,41.0,1666.0,68.848,28.691,0.000,2.461,White
40688,230311425,01/31/2023 06:16:06 PM,"500 S 1ST ST,MKE",2,PATROL,Assignment Completed,43.008533,-87.911185,5.507902e+10,016600,...,586.0,377.0,19.0,1014.0,1996.0,29.359,18.888,0.952,50.802,Hispanic
40751,230311327,01/31/2023 05:25:43 PM,"2302 W CAPITOL DR,MKE",5,PATROL,Assignment Completed,43.089978,-87.941111,5.507900e+10,004200,...,207.0,3742.0,0.0,195.0,4139.0,4.995,90.299,0.000,4.706,Black
40770,230311300,01/31/2023 05:11:25 PM,3200 N 27TH ST,5,PATROL,Assignment Completed,43.077573,-87.947132,5.507901e+10,006500,...,55.0,2289.0,17.0,114.0,2475.0,2.222,92.485,0.687,4.606,Black


In [41]:
tracts_w_patrols.groupby(['Prodominent Tract Population']).size().sort_values(ascending = False) / df2.groupby(['Prodominent Tract Population']).size().sort_values(ascending = False)

Prodominent Tract Population
White       0.522727
Black       0.876289
Hispanic    0.894737
dtype: float64

In [40]:
tracts_w_patrols = df2[df2['Tract'].isin(patroldata.groupby(['Tract']).size().sort_values(ascending = False).index.tolist())]
tracts_w_patrols

,GEOID,Tract,Geometry,Estimated Population,Male Population,Female Population,Median Age,Median Income,White Population,Black Population,Native Population,Hispanic Population,Income Poverty Level,Percent White,Percent Black,Percent Native,Percent Hispanic,Prodominent Tract Population
1,008800,55079008800,POLYGON ((-65976.74531288422 4791487.360095243...,1620.0,795.0,825.0,32.5,16713.0,90.0,1498.0,4.0,28.0,1620.0,5.556,92.469,0.247,1.728,Black
2,008500,55079008500,POLYGON ((-64585.292365803034 4791763.78518607...,1540.0,756.0,784.0,29.5,19546.0,6.0,1517.0,17.0,0.0,1540.0,0.390,98.506,1.104,0.000,Black
4,008900,55079008900,POLYGON ((-66590.0980055522 4791541.1265165135...,1548.0,724.0,824.0,21.4,12663.0,95.0,1268.0,0.0,185.0,1444.0,6.137,81.912,0.000,11.951,Black
5,009000,55079009000,POLYGON ((-67253.65211980906 4790705.913242541...,2008.0,740.0,1268.0,22.3,16311.0,64.0,1623.0,0.0,321.0,1966.0,3.187,80.827,0.000,15.986,Black
6,009100,55079009100,POLYGON ((-67849.48754489853 4791240.738696917...,1930.0,905.0,1025.0,31.3,13194.0,154.0,1531.0,0.0,245.0,1930.0,7.979,79.326,0.000,12.694,Black
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,002600,55079002600,POLYGON ((-67066.56444233644 4795193.581384823...,2671.0,1103.0,1568.0,29.1,18661.0,247.0,2325.0,27.0,72.0,2663.0,9.247,87.046,1.011,2.696,Black
281,002800,55079002800,POLYGON ((-67761.96733339364 4796157.683783457...,2459.0,1169.0,1290.0,26.3,18570.0,147.0,2113.0,5.0,194.0,2387.0,5.978,85.929,0.203,7.889,Black
282,003000,55079003000,POLYGON ((-70195.59329969867 4796399.667200316...,4507.0,2163.0,2344.0,26.7,27378.0,417.0,3465.0,2.0,623.0,4507.0,9.252,76.880,0.044,13.823,Black
283,003100,55079003100,POLYGON ((-69395.83786693972 4795469.244458136...,3862.0,1719.0,2143.0,31.6,21779.0,478.0,2835.0,25.0,524.0,3845.0,12.377,73.408,0.647,13.568,Black


In [26]:
df2.groupby(['Prodominent Tract Population']).size().sort_values(ascending = False)

Prodominent Tract Population
White       176
Black        97
Hispanic     19
dtype: int64

In [44]:
print(max(df1["Date/Time"]))

04/01/2023 12:59:45 PM
